This notebook allows user to study departure and arrival delays of flights from NYC airports in an interactive manner.

From https://plotly.com/python/figurewidget-app/

Unfortunately, it was not possible to use the tutorial code "as is" due to what appears an issue with FigureWidget. However, it is still possible to create an interactive plot using ipywidgets.interact.

# Load libraries, data

In [ ]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
from ipywidgets import interact

In [ ]:
df = pd.read_csv('../input/nyc-flights-2013/nyc_flights.csv')
df = df.drop(df.columns[[0]], axis=1)

# Define widgets that will allow us to interact with the plot

In [ ]:
#Do we filter data for a particular month?
use_month_widget = widgets.Checkbox(
    description='Focus on particular month: ',
    value=True,
)

#If so, which one?
month_widget = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

#Name of the carrier
carrier_widget = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

#Origin airport
origin_widget = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Create a figure which we will update
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.Figure(
            data=[trace1, trace2],
            layout=go.Layout(
                        title=dict(text='NYC FlightDatabase'),
                        barmode='overlay'
                        )
            )

# Interactive plot

In [ ]:
#Make sure the selected choices are sensible
def validate(origin, carrier):
    if origin in df['origin'].unique() and carrier in df['carrier'].unique():
        return True
    else:
        return False

In [ ]:
#Allows us to interactively change the selection parameters
@interact(
    month     = month_widget, 
    carrier   = carrier_widget, 
    origin    = origin_widget, 
    use_month = use_month_widget
)
#We must provide default values, otherwise the widgets do not show up
def interactive_plot(use_month = 'True', month = 1, carrier = 'DL', origin = 'LGA'):
    
    #Assert good choices were selected
    if validate(origin, carrier):
        
        #Filter by month?
        if use_month:
            filter_list = [
                            m and c and o
                            for 
                            m, c, o
                            in
                            zip(df['month'] == month, df['carrier'] == carrier,
                               df['origin'] == origin)
                          ]
        else:
            filter_list = [
                            c and o 
                            for 
                            c, o
                            in
                            zip(df['carrier'] == 'DL', df['origin'] == origin)
                          ]
            
        #Select data according to our choices
        temp_df = df[filter_list]
            
        #Results to plot
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'
        g.show()